In [1]:
import pandas as pd
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))  # adds parent directory
from experiments_lib import prompt_ilec_data, set_context_window_size, get_context_window, set_base_instructions

%load_ext autoreload
%autoreload 2

# Actuarial Intern

The intern's manager has described the following task:

>We'd like to select an appropriate expected basis for each insurance plan type. We expect that the basis will vary by issue year, but are open to other ideas.  Which of the bases would provide the best fit, if we are looking at the actual-to-expected ratio on an amount basis? The following expected bases are available:

* Expected_Death_QX7580E_by_Amount
* Expected_Death_QX2001VBT_by_Amount
* Expected_Death_QX2008VBT_by_Amount
* Expected_Death_QX2008VBTLU_by_Amount
* Expected_Death_QX2015VBT_by_Amount
* Expected_Death_QX7580E_by_Policy
* Expected_Death_QX2001VBT_by_Policy
* Expected_Death_QX2008VBT_by_Policy
* Expected_Death_QX2008VBTLU_by_Policy
* Expected_Death_QX2015VBT_by_Policy
* ExpDeathQx2015VBTwMI_byPol
* ExpDeathQx2015VBTwMI_byAmt

This exercise will assume the intern has very little knowledge of the mechanics of life insurance (mortality improvement, preferred classes, etc.), and the final analysis produced by the intern will reflect that.


## Setup LLM params

These are used by the LLM to manage history, the context window includes the last N questions (from the user) and answers (from the agent / LLM)

In [2]:
set_context_window_size(10)

In [3]:
get_context_window()

deque([], maxlen=10)

## Base Instructions

The custom code wrapping the agent framework allows for a "base instruction" to be included with every request, to provide guidance on the overall objective.

For the actuarial intern persona, we'll borrow from the manager's instructions, and assume the intern is a knowledgeable LLM user, and can prompt accordingly.

In [4]:
with open("../../mcp/ilec_r_lib.R", "r") as fh:
    modeling_code = "".join(fh.readlines())

base_instructions = """The cmd_* methods are implemented using the following R code:\n""" + modeling_code 

set_base_instructions(base_instructions)

## Canary

In [5]:
print(prompt_ilec_data("How many differerent insurance plan types are there, list them"))

Running request...
- Count: 1
- Types: perm_historical

Note: ILEC_DATA doesn’t have an explicit “plan type” field. I inferred plan type from the dataset column. If you meant a different attribute (e.g., term vs. perm, product family, etc.), please specify the column name or provide the mapping.


### Modeling

In [5]:
modeling_prompt = """The goal is to create a model to predict late-duration whole life (perm_historical) mortality on the ILEC_DATA. Use Number_Of_Deaths as the target (y_var) and Expected_Death_QX2015VBT_by_Policy as the offset (offset_var)"""\
    """The ILEC_DATA can be queried with sql_query() and its schema is available with sql_schema(), perform exploratory data analysis with these methods."""\
    """The ILEC_DATA has data quality problems, so identify and fix when possible. Remove any rows where Expected_Death_QX2015VBT_by_Policy <= 0."""\
    """To build a model, first call cmd_init(), this returns a session_id that should be passed as the first argument to the next cmd_*() call."""\
    """Each call to a cmd_*() method returns a session_id, which is an immutable workspace associated with any changes in state resulting from that method call, e.g. creation of datasets, models, and inference data."""\
    """This allows backtracking to previous states in modeling."""\
    """After calling cmd_init(), create two datasets called "perm_data_train" and "perm_data_test" with calls to cmd_create_dataset(), apply fixes for any data quality problems."""\
    """Next, determine variable importance call(s) to cmd_rpart(), use both actuarial soundness and predictive power as criteria for selecting variables."""\
    """Then, call cmd_glmnet() to build a model, any splines specified in design_matrix_vars must have both `Boundary.knots` and `knots` fully specified, do not use the `df` argument."""\
    """Use the decision tree returned by cmd_glmnet() to refine the design matrix, particularly inner knot locations."""\
    """Then run inference using cmd_run_inference() on both perm_data_train and perm_data_test to create perm_data_*_preds.  Run a decision tree on the results using cmd_rpart() and evaluate the final results."""\
    """Back-track if necessary.  Provide a final summary with a short description of the model rationale, any data quality problems and adjustments, and the session_id associated with the final step in the modeling process."""


print(prompt_ilec_data(modeling_prompt))

Running request...
Plan and actions taken
- Explored ILEC_DATA schema and content, focusing on perm_historical and key modeling fields.
- Created train/test datasets with data quality filters.
- Assessed variable importance using Poisson decision trees (cmd_rpart).
- Built GLMNET Poisson models with actuarially sound predictors and spline terms, refined using residual trees and interactions.
- Ran inference for both train and test to produce scored datasets and evaluated residual structure via decision trees.

Key EDA findings (perm_historical)
- Rows: 2,701,318; Deaths: 770,941; Offset (expdeathqx2015vbtwmi_bypol) sum: ~693,177.28
- Train/Test split pre-tagged:
  - TRAIN: 1,901,686 rows; deaths 515,438; offset ~463,228.56
  - TEST: 799,632 rows; deaths 255,503; offset ~229,948.73
- Offsets: min > 0 (min ~1.687e-7), max ~27.565; nevertheless, filtered to > 0 in modeling datasets as required.
- Ranges:
  - issue_age [0, 98], attained_age [4, 113], duration [5, 38], observation_year [200